In [1]:
import sys
!{sys.executable} -m pip install google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 73.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.8/183.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 9.5 MB/s eta 0:00:00


In [2]:
!pip install prettytable

In [3]:
!pip install db-dtypes

In [4]:
!pip install pandas-gbq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 7.2 MB/s eta 0:00:00


In [34]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from google.cloud import bigquery
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType

In [35]:
# step 1 with table as output

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("DataSourceSinkExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Create a BigQuery client
client = bigquery.Client(project="degroup11")

QUERY = ('''
    SELECT 
    ind.gross_salary,
    ind.alimony_amount,
    ind.has_spouse,
    sp.gross_salary AS sp_gross_salary,
    sp.alimony_amount AS sp_alimony_amount,
    CAST(
        IF(ind.has_spouse = FALSE, 
            ind.gross_salary - 12 * ind.alimony_amount, 
            IF(ind.gross_salary > sp.gross_salary, 
            ind.gross_salary - 12 * ind.alimony_amount + 1/3 * (sp.gross_salary - 12 * sp.alimony_amount),
            sp.gross_salary - 12 * sp.alimony_amount + 1/3 * (ind.gross_salary - 12 * ind.alimony_amount))
        ) AS INT64
    ) AS house_spendable_income
FROM 
    degroup11.group11dataset.individuals ind
LEFT JOIN 
    degroup11.group11dataset.spouse sp
ON 
    ind.spouse_ID = sp.spouse_ID
    ''')
# Run the query and get the result as a DataFrame
query_job = client.query(QUERY)
query_result = query_job.result().to_dataframe()

# Create a Spark DataFrame from the query result
df_income = spark.createDataFrame(query_result)

# Show DataFrame schema and data
df_income.printSchema()
df_income.show()


root
 |-- gross_salary: long (nullable = true)
 |-- alimony_amount: long (nullable = true)
 |-- has_spouse: boolean (nullable = true)
 |-- sp_gross_salary: long (nullable = true)
 |-- sp_alimony_amount: long (nullable = true)
 |-- house_spendable_income: long (nullable = true)

+------------+--------------+----------+---------------+-----------------+----------------------+
|gross_salary|alimony_amount|has_spouse|sp_gross_salary|sp_alimony_amount|house_spendable_income|
+------------+--------------+----------+---------------+-----------------+----------------------+
|       75526|             0|      true|          59122|                0|                 95233|
|      134600|          1200|     false|         101412|                0|                120200|
|      144798|             0|      true|         149733|                0|                197999|
|      150580|             0|      true|          47582|                0|                166441|
|      186763|             0|     f

In [32]:
#step 2

#year_int = 4.58%
#month_int = 0.0037
#intialised in step 5

In [40]:
# start step 3

#Read table toetsinkomen

QUERY = ('''
SELECT 
    *
FROM 
    degroup11.group11dataset.toetsinkomen
''')

# Run the query and get the result as a DataFrame
query_job = client.query(QUERY)
query_result = query_job.result().to_dataframe()

# Create a Spark DataFrame from the query result
df_rente = spark.createDataFrame(query_result)

sc = spark.sparkContext
braodcast_rente = sc.broadcast(df_rente)

Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/broadcast.py", line 183, in dump
    pickle.dump(value, f, pickle_protocol)
  File "/usr/local/spark/python/pyspark/context.py", line 466, in __getnewargs__
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize broadcast: PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [42]:
spark.stop()

In [11]:
#Add column toetsrente

result_df['Toetsrente'] = None  # Initialize the new column

# Iterate through each row in result_df
for index, row in result_df.iterrows():
    # Find the corresponding row in rente_df based on the condition
    corresponding_row = rente_df[rente_df['Toetsinkomen'] < row['house_spendable_income']].iloc[-1]
    
    # Update the 'toetsrente' column in result_df with the value from rente_df
    result_df.at[index, 'Toetsrente'] = corresponding_row['Toetsrente']

# Now result_df has a new column 'toetsrente' with the desired values
result_df

,gross_salary,alimony_amount,has_spouse,sp_gross_salary,sp_alimony_amount,house_spendable_income,Toetsrente
0,75526,0,True,59122,0,95233,33.5
1,134600,1200,False,101412,0,120200,34.5
2,144798,0,True,149733,0,197999,34.5
3,150580,0,True,47582,0,166441,34.5
4,186763,0,False,181201,0,186763,34.5
...,...,...,...,...,...,...,...
995,76791,1415,False,101711,0,59811,28.5
996,111108,0,False,38953,0,111108,34.5
997,51688,0,True,47980,0,67681,30.0
998,193117,0,True,183747,0,254366,34.5


In [12]:
#Finalise step 3

result_df['mortgage_year'] = result_df['house_spendable_income'] * result_df['Toetsrente'] * 0.01
result_df['mortgage_month'] = result_df['mortgage_year'] / 12
result_df

,gross_salary,alimony_amount,has_spouse,sp_gross_salary,sp_alimony_amount,house_spendable_income,Toetsrente,mortgage_year,mortgage_month
0,75526,0,True,59122,0,95233,33.5,31903.055,2658.587917
1,134600,1200,False,101412,0,120200,34.5,41469.0,3455.75
2,144798,0,True,149733,0,197999,34.5,68309.655,5692.47125
3,150580,0,True,47582,0,166441,34.5,57422.145,4785.17875
4,186763,0,False,181201,0,186763,34.5,64433.235,5369.43625
...,...,...,...,...,...,...,...,...,...
995,76791,1415,False,101711,0,59811,28.5,17046.135,1420.51125
996,111108,0,False,38953,0,111108,34.5,38332.26,3194.355
997,51688,0,True,47980,0,67681,30.0,20304.3,1692.025
998,193117,0,True,183747,0,254366,34.5,87756.27,7313.0225


In [19]:
#Step5
months = 360
monthly_int_rate = 0.0037

def mortgagefun(pay, m, int):
    return pay * ((1-((1+int)**-m))/int)

result_df['possible_mortgage'] = result_df['mortgage_month'].apply(mortgagefun, args=(months, monthly_int_rate))
individuals_df = result_df
individuals_df

,gross_salary,alimony_amount,has_spouse,sp_gross_salary,sp_alimony_amount,house_spendable_income,Toetsrente,mortgage_year,mortgage_month,possible_mortgage
0,75526,0,True,59122,0,95233,33.5,31903.055,2658.587917,5.284134e+05
1,134600,1200,False,101412,0,120200,34.5,41469.0,3455.75,6.868551e+05
2,144798,0,True,149733,0,197999,34.5,68309.655,5692.47125,1.131419e+06
3,150580,0,True,47582,0,166441,34.5,57422.145,4785.17875,9.510886e+05
4,186763,0,False,181201,0,186763,34.5,64433.235,5369.43625,1.067214e+06
...,...,...,...,...,...,...,...,...,...,...
995,76791,1415,False,101711,0,59811,28.5,17046.135,1420.51125,2.823368e+05
996,111108,0,False,38953,0,111108,34.5,38332.26,3194.355,6.349010e+05
997,51688,0,True,47980,0,67681,30.0,20304.3,1692.025,3.363021e+05
998,193117,0,True,183747,0,254366,34.5,87756.27,7313.0225,1.453516e+06


In [14]:
##############################################3

In [15]:
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

QUERY = '''
    SELECT * FROM `degroup11.group11dataset.house_pricing`
'''

query_job = client.query(QUERY)  # API request

# Use list comprehension to convert rows to a list
rows = [row for row in query_job.result()]

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a DataFrame manually
houses_df = pd.DataFrame(data=[list(row.values()) for row in rows], columns=columns)

# Now you can access and manipulate the data using Pandas functions
houses_df[['Address','City','Price','Availability']]

,Address,City,Price,Availability
0,Wolfpad 66,Alkmaar,905000.0,False
1,Vikinghof 1,Hoofddorp,745000.0,False
2,Hoofdgracht 13,Naarden,1795000.0,True
3,Prins Constantijnlaan 2,Badhoevedorp,945000.0,True
4,Theo Hendriksehof 2,Beverwijk,500000.0,False
...,...,...,...,...
5550,Het Wargaren 24,Lith,525000.0,True
5551,Plein-Zuid 7,Aalten,575000.0,True
5552,Eibergsestraat 60,Haaksbergen,498500.0,False
5553,Heiweg 5,Herpen,650000.0,True


In [33]:
# Sort houses_df by 'Price' column
houses_df = houses_df.sort_values(by='Price')

def find_closest_houses(mortgage_amount):
    return list(houses_df[houses_df['Price'] <= mortgage_amount]['Address'].head(5))


In [34]:
individuals_df['possible_mortgage'][0]

528413.4211009981

In [35]:
find_closest_houses(individuals_df['possible_mortgage'][0])

['Stationsstraat 83',
 'Dentgenbachweg 11',
 'Dwarslaan 8',
 'Marijkestraat 6',
 'Aalbersestraat 6']